In [1]:
# This is Main function.
# Extracting streaming data from Twitter, pre-processing, and loading into MySQL
import credentials # Import api/access_token keys from credentials.py
import settings # Import related setting constants from settings.py 

import re
import tweepy
import mysql.connector
import pandas as pd
from textblob import TextBlob
# Streaming With Tweepy 
# http://docs.tweepy.org/en/v3.4.0/streaming_how_to.html#streaming-with-tweepy


# Override tweepy.StreamListener to add logic to on_status
class MyStreamListener(tweepy.StreamListener):
    '''
    Tweets are known as “status updates”. So the Status class in tweepy has properties describing the tweet.
    https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object.html
    '''
    
    def on_status(self, status):
        '''
        Extract info from tweets
        '''
        
        if status.retweeted:
            # Avoid retweeted info, and only original tweets will be received
            return True
        # Extract attributes from each tweet
        id_str = status.id_str
        created_at = status.created_at
        text = deEmojify(status.text)    # Pre-processing the text  
        sentiment = TextBlob(text).sentiment
        polarity = sentiment.polarity
        subjectivity = sentiment.subjectivity
        
        user_created_at = status.user.created_at
        user_location = deEmojify(status.user.location)
        user_description = deEmojify(status.user.description)
        user_followers_count =status.user.followers_count
        longitude = None
        latitude = None
        if status.coordinates:
            longitude = status.coordinates['coordinates'][0]
            latitude = status.coordinates['coordinates'][1]
            
        retweet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        print(status.text)
        print("Long: {}, Lati: {}".format(longitude, latitude))
        
        # Store all data in MySQL
        if mydb.is_connected():
            mycursor = mydb.cursor()
            sql = "INSERT INTO {} (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(settings.TABLE_NAME)
            val = (id_str, created_at, text, polarity, subjectivity, user_created_at, user_location, \
                user_description, user_followers_count, longitude, latitude, retweet_count, favorite_count)
            mycursor.execute(sql, val)
            mydb.commit()
            mycursor.close()
    
    
    def on_error(self, status_code):
        '''
        Since Twitter API has rate limits, stop srcraping data as it exceed to the thresold.
        '''
        if status_code == 420:
            # return False to disconnect the stream
            return False


In [2]:
def clean_tweet(self, tweet): 
    ''' 
    Use sumple regex statemnents to clean tweet text by removing links and special characters
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) \
                                |(\w+:\/\/\S+)", " ", tweet).split()) 
def deEmojify(text):
    '''
    Strip all non-ASCII characters to remove emoji characters
    '''
    if text:
        return text.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

In [3]:
mydb = mysql.connector.connect(
    host="localhost",
    user="twitter",
    passwd="project",
    database="twitter",
    charset = 'utf8'
)
if mydb.is_connected():
    '''
    Check if this table exits. If not, then create a new one.
    '''
    mycursor = mydb.cursor()
    mycursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_name = '{0}'
        """.format(settings.TABLE_NAME))
    if mycursor.fetchone()[0] != 1:
        mycursor.execute("CREATE TABLE {} ({})".format(settings.TABLE_NAME, settings.TABLE_ATTRIBUTES))
        mydb.commit()
    mycursor.close()
    



In [4]:
auth  = tweepy.OAuthHandler(credentials.API_KEY, credentials.API_SECRET_KEY)
auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener = myStreamListener)
myStream.filter(languages=["en"], track = settings.TRACK_WORDS)
# Close the MySQL connection as it finished
# However, this won't be reached as the stream listener won't stop automatically
# Press STOP button to finish the process.
mydb.close()

RT @SirJambavan: Okay .. I have had enough of the fake news and photos and blaming of Kumbh for persistently high Covid numbers of Maharash…
Long: None, Lati: None
Covid reality, numbers explained. 
Country: France. 
100,000 death since February 2020.
➡️ &lt; 50s 1.3% = 951 deaths… https://t.co/gD86HdPtOV
Long: None, Lati: None
RT @jellybeansoup3: #postponejee2021 please support and trend. Help the students. We are being forced to give exam in covid hotspots
Long: None, Lati: None
RT @wayneradiotv: this is what anti vaxxers think getting a dose of the covid shot is like https://t.co/1Sl4IVoONb
Long: None, Lati: None
RT @TheRobustRascal: Scenes from Powai, Mumbai

Look how poor fruit seller is harassed by BMC people and his fruits are destroyed.

No acti…
Long: None, Lati: None
RT @NBCNews: In Brazil, an alarmingly high number of babies and children are dying of Covid-19. https://t.co/Tb1C5rxW0B
Long: None, Lati: None
Scientifically, in order for countries to return to a normal state in

Crowdsourced Testing Market Share 2021 COVID-19 Pandemic Impact, Trends, Competitive Landscape and Segmentation by… https://t.co/h01cHdsglQ
Long: None, Lati: None
Did Didi ask these people to stay at home and warned them of the grave dangers of this pandemic?
Long: None, Lati: None
RT @Neerajkundan: Dear All @nsui members.

Let's start #LadengeAurJeetenge campaign once again. 

◆Prepare a list of covid recovered patien…
Long: None, Lati: None
Rapid Covid testing in England may be scaled back over false positives https://t.co/vk2Hh261EP
Long: None, Lati: None
RT @chappan_kilo: My colleague's relative requires COVID plasma (B+) in Max Hospital, Shalimar Bagh, New Delhi.
Please RT it to your networ…
Long: None, Lati: None
RT @nanditathhakur: My father Shri Omkar Singh Thakur JI lost his battle against Covid.. Thank You all for youR wishes, but something is de…
Long: None, Lati: None
Madagascar Postpones Schools Reopening Over Covid https://t.co/uK2M1QqbUe
Long: None, Lati: None
RT @a_bria

RT @bainjal: Prayers for Gujarat
Long: None, Lati: None
RT @JasmineHayer: It took 11 months for me to FINALLY get a diagnosis for #LongCovid. Anxiety wasn't the cause, it was blood clots in my lu…
Long: None, Lati: None
RT @akhil_3101: Any COVID recovered people willing to donate plasma around vijayawada.
This is for my friend’s parents.
Please retweet 🙏🏻 h…
Long: None, Lati: None
RT @DanielHugill: I had seen photos of the Covid Memorial Wall but I had no idea how long it is.

This morning I cycled along it.

RIP ❤️ h…
Long: None, Lati: None
“St Basil's has received nearly $73 million in taxpayer funding over the past eight years.

He said a manager told… https://t.co/IhQoilxMiu
Long: None, Lati: None
RT @C4Ciaran: I asked @NicolaSturgeon about the decision to discharge COVID positive and untested patients into care homes in Scotland, whe…
Long: None, Lati: None
RT @ThomasReeseSJ: Check out this cartoon https://t.co/RUt4k373sj via @TheWeek
Long: None, Lati: None
RT @FinessaFudges: Pfiz

RT @ChaitanyaTOI: Amid shortage of beds in Mumbai Jumbo Covid centers &amp; Seven Hills Hospital have been real life savers. Jumbos haven’t ref…
Long: None, Lati: None
RT @amitmalviya: Some states have raised the issue of vaccine shortages. Vaccine manufacturers are facing problems in procuring adjuvants f…
Long: None, Lati: None
I'm the kind of guy who would rather get COVID instead of the vaccine. I'm not anti-vax, I just have a paralyzing f… https://t.co/yLZyBxKoO4
Long: None, Lati: None
RT @India_Policy: This doesn't seem correct.

On an average, over 25,000 people die in India everyday from non covid causes.

Even a 10% su…
Long: None, Lati: None
RT @Undergroundsar3: Another friend today tested positive for COVID after being fully vaccinated.  There's something NOT right here, folks!
Long: None, Lati: None
104-year-old woman released from hospital after 2nd bout with COVID-19 https://t.co/Z8Un4yOMTY #arkgynecology… https://t.co/yJX6u2ykDX
Long: None, Lati: None
RT @billbowtell: Fo

RT @AbhishBanerj: Two bits of fake news came:

(1) Someone with no PhD and no peer reviewed papers made tenured Harvard prof at non-existen…
Long: None, Lati: None
RT @propublica: Ohio juvenile court Judge Timothy Grendell thought coronavirus precautions were overblown, and made sure people knew it. In…
Long: None, Lati: None
Amit Shah, Chief Election Commissioner: Disallow all Mass Public Gatherings including Religious events and Politica… https://t.co/VNNJ4fYMla
Long: None, Lati: None
Question: Would OTHER anti-parasite medications work to inhibit, halt and/or treat #COVID?
Long: None, Lati: None
RT @AK4784877825: @PMO @CMOMaharashtra
Govt. Mandate! Drivers with App Based Cabs, Taxis, Autos to carry Covid Free Certificates! Very unfa…
Long: None, Lati: None
RT @TaxBeast: I think one of the things that prepped me for the Covid "6ft apart" rule was growing up playing DnD. All miniatures took up a…
Long: None, Lati: None
RT @HHSGov: Discover how long it takes to get full immunity after 

...

RT @DarrenPlymouth: Rapid Covid testing in England may be scaled back over false positives. Over 90% of positive results in areas with low…
Long: None, Lati: None
RT @Sougata_Mukh: Frantic activities of polling personnel in Bengal's hill town of Darjeeling ahead of 5th phase of polls scheduled to take…
Long: None, Lati: None
RT @jaicabajar: Sharing a COVID home care algorithm done by the UP College of Medicine Class of 1991! https://t.co/38Ot93GoHt
Long: None, Lati: None
RT @citizentvkenya: 'I was not invited to State House,' DP Ruto defends his move to go for Russia's Sputnik V COVID-19 vaccine
https://t.co…
Long: None, Lati: None
RT @Oye_Jahazi: Greta Thunberg has asked Govt to plant trees in COVID wards to overcome the oxygen shortage problem as trees release O2 and…
Long: None, Lati: None
RT @KagutaMuseveni: I will be addressing you later today on a number of issues and making some clarifications regarding the COVID-19 situat…
Long: None, Lati: None
Don’t inject #Remdesivir in you,

RT @THEJASCGOWDA: #savevtustudents 
As If everyone know Bus strike is going on and Covid-19/21 is blasting , So please postpone the exam an…
Long: None, Lati: None
Chlore in pools kills the covid virus...
Long: None, Lati: None
RT @JavedRanaa: There is a sudden rise in COVID cases throughout the country. Government must make a move before its too late.
Long: None, Lati: None
RT @SushantV11: @coolfunnytshirt I have seen pics of people breaking covid protocols during other gatherings also but none of them have twe…
Long: None, Lati: None
RT @latimeralder: UK Covid stats today: 16/4/21

- 48% with at least one vaccine jab

- 501 NHS workers for each 'with Covid' patient

For…
Long: None, Lati: None
@yagnaja Very insensitive in such covid times.
Long: None, Lati: None
RT @JamesMelville: A vaccinated person can still get Covid (but are likely to have reduced symptoms because of the vaccine). But they can s…
Long: None, Lati: None
RT @FaheemYounus: Treating Severe COVID at Home; No Hospital 

RT @thomasdolphin: We nearly got to this point in the UK, just so you know. We stopped using propofol infusions in theatres bc it was all b…
Long: None, Lati: None
RT @TheQuartering: Just tell me I can't cover covid.  Fine.  But at least make it a rule and retract the strike
Long: None, Lati: None
RT @tweettruth2me: No one saw this coming. 🙄 https://t.co/qJrwT3XOCP
Long: None, Lati: None
RT @ianbremmer: Risk of blood clots 

J&amp;J vaccine .00001%
Birth control pills: .05%
COVID infection: 16%

(CDC)
Long: None, Lati: None
RT @NandyAsmita: Hate to break it to you Bengal, neither of the two political parties who could possibly come to power gives a damn about t…
Long: None, Lati: None
RT @MonaHannaA: MI, shut down.  

From a nurse friend just now:

“I’m exhausted 
It’s crazy here 
I’m overwhelmed
125 people in ER
No room…
Long: None, Lati: None
RT @realnikhil11: #WhereisPM #postponejee2021 @DrRPNishank  Day by Day Situation is getting extremely critical. So many people tested covid…
Lo

People no longer care about the covid-19.
Long: None, Lati: None
To be honest it’s obvious. You cannot hide from a virus. It will find you in the end. It’s be practicing for millio… https://t.co/sgPuWQVHi9
Long: None, Lati: None
RT @wilsonchua: Might this explain how the new covid variants quickly took hold in the Philippines? https://t.co/tHvmZL5vDr https://t.co/Jn…
Long: None, Lati: None
RT @Qldaah: Pfizer offered the Australian govt its covid-19 vaccine on July 10. Nothing happened until the end of November when a deal was…
Long: None, Lati: None
RT @shikhar123stu: Thank so much for your support sonu sir...#cancelboardexam2021 #cancelboardexams2021 #CBSE  #SonuSood  #UPBoard
Long: None, Lati: None
RT @yogye: Did he even know what was happening in tge PMO let alone the country? The so called economist PM who brought the economy to it's…
Long: None, Lati: None
RT @bbclornag: Scotland’s stay local restrictions ease. ⁦@lomondtrossachs⁩ gearing up for visitors. Some here already, includi

@madball911 I'll bet. We all took a lot for granted before COVID. But in a few weeks you should be in the clear rig… https://t.co/8zG6d02GPH
Long: None, Lati: None
RT @htTweets: The Odisha government has again demanded at least 25 lakh doses of Covishield and Covaxin vaccines in a single supply

(repor…
Long: None, Lati: None
RT @ScotLithVAian: 5,800 Fully Vaccinated Americans Have Contracted COVID-19, 74 Dead: CDC
https://t.co/yXmteXWWru
Long: None, Lati: None
RT @anvith_kateel: The government is playing in life of the students .
1. No Transport Facility government bus
2.Exams in peak time of Covi…
Long: None, Lati: None
RT @shrishti827: Noida Gurdwara 18 Sector is performing a very noble act by providing cooked food to COVID affected persons who have no one…
Long: None, Lati: None
RT @sardesairajdeep: The one thing COVID reminds us: a virus knows no political boundaries. So whether it is BJP ruled Gujarat or oppn rule…
Long: None, Lati: None
RT @AmitsharmaGRENO: India is the fastest 